In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| default_exp network

In [ ]:
#| export
import socket
import json

In [ ]:
#| export
class Server():
    def __init__(self, host='localhost', port=6666 , buf_size=1024):
        self.buf_size = buf_size
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            s.bind((host, port))
            s.listen()
            print(f"Waiting for connection on {host}:{port}")
            self.connection, addr = s.accept()
            print(f"Connected by {addr}")
        self.open=True
            
    def get(self):
        data = self.connection.recv(self.buf_size)
        return data.decode()
    
    def put(self, data):
        d = str.encode(data)
        self.connection.sendall(d)

    def get_dict(self):
        data = self.connection.recv(self.buf_size)
        dict = eval(data.decode())
        return dict
    
    def put_dict(self, dict):
        json_object = json.dumps(dict, indent = 4) 
        d = str.encode(json_object)
        self.connection.sendall(d)

    def isOpen(self):
        return self.open

    def finish(self):
        self.open = False

    def close(self):
        self.open=False
        self.connection.close()
        print('Closed server connection')

In [ ]:
#| export
class Client():
    
    def __init__(self, host='localhost', port=6666 , buf_size=1024):
        self.buf_size = buf_size
        self.connection = socket.create_connection((host,port))
        self.open=True
    
    def get(self):
        data = self.connection.recv(self.buf_size)
        return data.decode()
    
    def put(self, data):
        d = str.encode(data)
        self.connection.sendall(d)

    def get_dict(self):
        data = self.connection.recv(self.buf_size)
        dict = eval(data.decode())
        return dict
    
    def put_dict(self, dict):
        json_object = json.dumps(dict, indent = 4) 
        d = str.encode(json_object)
        self.connection.sendall(d)

    def isOpen(self):
        return self.open

    def finish(self):
        self.open = False
        
    def close(self):
        self.connection.close()
        #print('Closed client connection')


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()